In [2]:
import pandas as pd
import numpy as np
import scanpy as sc
import sys
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import io

# Load all

In [13]:
dpath = "/scratch/indikar_root/indikar1/cstansbu/scpc_test/align_table/"

reference = 'GRCm39'
columns = [
    'read_name', 
    'mapping_quality',
]
mapq = 60

df = []

for f in os.listdir(dpath):

    if not reference in f:
        pass
    
    fpath = f"{dpath}{f}"
    tmp = pd.read_parquet(fpath, columns=columns)
    sample = f.replace(".csv", "")[0:2]
    cell = f.replace(".csv", "")[2:5]
    condition  = f.split(".")[2]

    # drop imperfect alignments
    tmp = tmp[tmp['mapping_quality'] == mapq]

    # group by the number of alignments per read
    tmp = tmp.groupby('read_name')['mapping_quality'].count().reset_index()
    tmp = tmp.rename(columns={'mapping_quality' : 'n_perfect_alignments'})

    tmp['sample'] = sample
    tmp['cell'] = cell
    tmp['reference'] = reference
    tmp['condition'] = condition

    df.append(tmp)

df = pd.concat(df)
print(f"{df.shape=}")
df

KeyboardInterrupt: 

In [ ]:
# clean up and re-format


In [34]:
outpath = "/scratch/indikar_root/indikar1/cstansbu/scpc_test/analysis/excel_reports/alignment_summary.xlsx"
with pd.ExcelWriter(outpath, engine='xlsxwriter') as writer:

    # save each sample to a new tab
    for sample_names, group in df.groupby(['sample', 'reference']):
        sample, ref = sample_names
        group.to_excel(writer, sheet_name=f"Sample_{sample}_Ref_{ref}", index=True)

print(f'saved: {outpath}')

saved: /scratch/indikar_root/indikar1/cstansbu/scpc_test/analysis/excel_reports/alignment_summary.xlsx


In [3]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
tmp.agg(
    total_reads = ('read_name', 'nunique'),
    total_alignments = ('fragment_index', 'count'),
).head()

In [ ]:
    tmp['is_mapq_ge_20'] = tmp['mapping_quality'] > 20
    tmp['is_mapq_ge_50'] = tmp['mapping_quality'] > 50

    tmp = tmp.groupby('read_name').agg(
        total_alignments = ('fragment_index', 'count'),
        n_map_ge_20 = ('is_mapq_ge_20', 'sum'),
        n_map_ge_50 = ('is_mapq_ge_50', 'sum'),
        mean_align_base_quality = ('mean_align_base_quality', 'mean'),
        median_align_base_quality = ('mean_align_base_quality', 'median'),
        mean_mapq = ('mapping_quality', 'mean'),
        median_mapq = ('mapping_quality', 'median'),
        mean_alignment_score = ('alignment_score', 'mean'),
        median_alignment_score = ('alignment_score', 'median'),
        n_chroms = ('chrom', 'nunique'),
    ).reset_index()

    tmp['is_higher_order_candidate'] = tmp['n_map_ge_20'] > 2

In [ ]:
break